In [1]:
%cd ..

/data/home/eak/learning/llm_finetuning/specializing-llm-telecom


In [2]:
import os
import json

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
training = json.load(open("data/zindi_data/TeleQnA_training.json"))
testing = json.load(open("data/zindi_data/TeleQnA_testing1.json"))
extra_dataset = json.load(open("data/TeleQnA.json"))

len(training), len(testing), len(extra_dataset)

(1461, 366, 10000)

In [4]:
training = pd.DataFrame.from_dict(training).T
testing = pd.DataFrame.from_dict(testing).T
extra_dataset = pd.DataFrame.from_dict(extra_dataset).T

In [5]:
training.sample(7)

,question,option 1,option 2,option 3,option 4,answer,explanation,category,option 5
question 3103,What information is incorporated into the CDRs...,Charging events generated by CTF.,IOI identities received in the session signall...,Operator identification information used for IOI.,Addresses and identifiers of IMS NEs and appli...,option 2: IOI identities received in the sessi...,IOI identities received in the session signall...,Standards specifications,Correlation information for the same resource ...
question 1707,How does the scheduler assign resources betwee...,Based on the QoS requirements of each UE and a...,Based on the radio conditions at the UE identi...,Based on the UE buffer status,Based on the power headroom reports,option 1: Based on the QoS requirements of eac...,The scheduler assigns resources between UEs ta...,Standards specifications,Based on the measurements of a UEs radio envir...
question 6452,What is the purpose of ACDC (Application Speci...,To prevent overload of the access network and/...,To control access to network provided services...,To allow/prevent new access attempts from part...,To differentiate between CS and PS domains,option 3: To allow/prevent new access attempts...,The purpose of ACDC is to allow/prevent new ac...,Standards specifications,To specify the categories of subscribers who a...
question 6617,What is the purpose of the Recovery/Restoratio...,To expedite service recovery and restoration a...,To provide priority treatment for supplementar...,To enforce policy control for MPS application ...,To prioritize handover of MPS voice calls to CS,option 1: To expedite service recovery and res...,The Recovery/Restoration functionality in the ...,Standards specifications,To support MPS when roaming outside the home c...
question 6721,A UE monitors PDCCH for scheduling PDSCH recep...,Unicast and multicast,Broadcast and multicast,Unicast and broadcast,Multicast and SPS,option 4: Multicast and SPS,A UE monitors PDCCH for scheduling PDSCH recep...,Standards specifications,NaN
question 7278,How is V2X authorization between the UE and th...,Over the V1 interface,Over the V2 interface,Over the V3 interface,Over the V4 interface,option 3: Over the V3 interface,V2X authorization between the UE and the V2X C...,Standards overview,Over the V5 interface
question 4547,What is the purpose of the ECM Work Item and t...,To provide management provisions and solutions...,To support dynamic availability of Edge and EAS,To define the architecture for enabling Edge A...,To specify the security aspects of application...,option 1: To provide management provisions and...,The ECM Work Item and the resulting specificat...,Standards overview,To support service provisioning for UE


In [6]:
training["question"].str.split(" ").apply(len).describe()

count    1461.000000
mean       15.451061
std         4.066380
min         6.000000
25%        13.000000
50%        15.000000
75%        18.000000
max        35.000000
Name: question, dtype: float64

In [7]:
training["question"].apply(lambda x: len(x) / 3).describe()

count    1461.000000
mean       31.413872
std         8.424893
min        12.333333
25%        25.333333
50%        30.666667
75%        36.333333
max        67.666667
Name: question, dtype: float64

In [8]:
training.columns

Index(['question', 'option 1', 'option 2', 'option 3', 'option 4', 'answer',
       'explanation', 'category', 'option 5'],
      dtype='object')

In [9]:
columns = ['question', 'option 1', 'option 2', 'option 3', 'option 4', 'option 5']

def draw_stats(data: pd.DataFrame):
	for column in columns:
		raw = data[column][~data[column].isna()]
		i = raw.str.split(" ").apply(len).max()
		j = raw.apply(lambda x: len(x) / 3).max()
		k = raw.str.split(" ").apply("".join).apply(len).max() // 3
		print(column, i, j, k)

In [10]:
draw_stats(training)

question 35 67.66666666666667 58
option 1 49 115.0 99
option 2 49 115.0 99
option 3 49 115.0 99
option 4 49 115.0 99
option 5 49 115.0 99


In [11]:
draw_stats(testing)

question 32 68.33333333333333 59
option 1 24 54.0 46
option 2 25 53.333333333333336 45
option 3 22 42.0 37
option 4 24 49.666666666666664 42
option 5 25 45.333333333333336 38


In [12]:
draw_stats(extra_dataset)

question 38 98.33333333333333 86
option 1 49 115.0 99
option 2 49 115.0 99
option 3 49 115.0 99
option 4 49 115.0 99
option 5 49 115.0 99


In [13]:
len("option 2: To configure the MFAF to stop mapping data or analytics received by the MFAF to out") / 3

31.0

In [14]:
len("".join("option 2: To configure the MFAF to stop mapping data or analytics received by the MFAF to out".split(" "))) / 3

25.333333333333332